### **This notebook presents a formulation and solution of the following operations research question**.

I'm using the PYOMO framework and the GLPK solver.

<a href="https://colab.research.google.com/github/anapdinizm/operations-research-problems/blob/main/notebooks/Question_3_Parking_lines.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://raw.githubusercontent.com/anapdinizm/operations-research-problems/main/notebooks/Question_3_Parking_lines.ipynb"><img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

In [1]:
# Installing necessary packages
!apt-get install libglpk-dev
!apt-get install -y -qq glpk-utils
!pip install glpk
!pip install pyomo

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3 libglpk40 libgmp-dev
  libgmpxx4ldbl libgraphblas-dev libgraphblas6 libklu1 libldl2 libmetis5 libmongoose2 librbio2
  libsliplu1 libspqr2 libsuitesparse-dev libsuitesparseconfig5 libumfpack5
Suggested packages:
  libiodbc2-dev gmp-doc libgmp10-doc libmpfr-dev
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3 libglpk-dev libglpk40
  libgmp-dev libgmpxx4ldbl libgraphblas-dev libgraphblas6 libklu1 libldl2 libmetis5 libmongoose2
  librbio2 libsliplu1 libspqr2 libsuitesparse-dev libsuitesparseconfig5 libumfpack5
0 upgraded, 23 newly installed, 0 to remove and 35 not upgraded.
Need to get 23.5 MB of archives.
After this operation, 174 MB of additional disk space will be used.
Get:1 http://archive.ubun

## **Question 3**

### **STATEMENT**

Along a certain street in the city of Belo Horizonte, cars can be parked on both
sides. One person, who lives at number 1, is organizing a party for around 30 people, who will arrive in 15 cars. The length of the i − th car is λᵢ, expressed in meters as follows:


| i | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| λᵢ  | 4 | 4.5 | 5 | 4.1 | 2.4 | 5.2 | 3.7 | 3.5 | 3.2 | 4.5 | 2.3 | 3.3 | 3.8 | 4.6 | 3 |

Figure 3.1: Problem Instance

In order to avoid bothering the neighbours, this person would like **to arrange the parking on both sides of the street so that the length of the street occupied by his friends’ cars should be minimum**. Give a metaheuristic to solve the problem using python language. The code should be commented.

How does the program change if on exactly one of the street sides the cars should not occupy more than 15m?

### **SOLUTION**

In [2]:
#Importing necessary libraries
import pandas as pd
from pyomo.environ import *
import matplotlib.pyplot as plt
from IPython.display import display
from matplotlib.lines import Line2D

#### **Data**

In [3]:
CAR_LENGTH = [4, 4.5, 5, 4.1, 2.4, 5.2, 3.7, 3.5, 3.2, 4.5, 2.3, 3.3, 3.8, 4.6, 3]
#cenary one, umlimited capacity, without an upper bound for capacity
CAPACITY_1 = [1000, 1000]
#cenary two, capacity of 15m for one side of the street
CAPACITY_2 = [1000, 15]

#### **Deterministic problem**

##### **Model**

**Variables**

In this problem, the decision variable X indicates whether a car ($i$) is parked on side ($j$). The variable Y indicates whether the side $j$ was ocuppied or not by a car.

\begin{align*}
  X_{i,j} =
 \begin{cases}
      1, & \text{if car } i \text{ was parked at a specific side } j\\
      0, & \text{otherwise}
 \end{cases}
\end{align*}

\begin{align*}
 Y_j =
 \begin{cases}
      1, & \text{if side } j \text{ was used} \\
      0, & \text{otherwise}
 \end{cases}
\end{align*}

**Objective function**

For this problem, the objective is minimize the length of street occupied by cars. As we have both sides, minimizing just the sum of length will not give the proper objective, minimizing just the length. To do that minimizing the difference between both sides of street occupied wil help have the miminum of street with cars.

\begin{align*}
    \text{Minimize} \quad & \sum_{i \in I}{l_i X_{i,0}} - {l_i X_{i,1}} \\
\end{align*}

**Constraints**

The parameter $lᵢⱼ$ is the ength per car  $i$, side park $j$ and $cⱼ$ is the capacity per street side $j$ (left or right). The first constraint considers the lmit capacity per street side.

\begin{align*}
    \text{subject to}
    \quad & \sum_{i \in I}\sum_{j \in J} {l_{i} X_{i,j}} \leq c_j Y_j & \forall ~i \in I, \forall ~j \in J\\
    \quad & \sum_{i \in I}{l_i X_{i,0}} - {l_i X_{i,1}} \ge 0 & \forall ~i \in I \\
    \quad & \sum_{j \in J} {X_{i,j}} = 1 & \forall ~i \in I, \forall ~j \in J\\
    \quad & X_{i,j} \in \left \{ 0, 1 \right \} & \forall ~i \in I, \forall ~j \in J\\
    \quad & Y_{j} \in \left \{ 0, 1 \right \} & \forall ~j \in J\\
\end{align*}



##### **Code**

In [64]:
def capacity_restriction(m, j):
  '''
  This function represents the constraints of the road side capacity.
  The input isthe lenght of cars and the capacity per side of the road.
  '''
  temp = sum(CAR_LENGTH[i] * m.x[i,j] for i in m.I) - CAPACITY_1[j] * m.y[j]
  if type(temp) is int:    # This will be true if the sum was empty
    return Constraint.Skip
  return temp <= 0

def diff_pos(m, CAR_LENGTH):
  '''
  This function represents the constraints that guarantee that the difference in
  length between the sides is greater than or equal to zero, i.e. positive.
  '''
  temp = sum(CAR_LENGTH[i] * m.x[i,0] for i in m.I ) - sum(CAR_LENGTH[i] * m.x[i,1] for i in m.I )
  if type(temp) is int:    # This will be true if the sum was empty
    return Constraint.Skip
  return temp >= 0

def side_selected(m, i):
  '''
  This function represents the constraints that guarantee that a car will be parked
   only on one side. Can't be parked on both sides.
  '''
  temp = sum(m.x[i,j] for j in m.J)
  if type(temp) is int:    # This will be true if the sum was empty
    return Constraint.Skip
  return temp == 1

In [69]:
def parking_length():
  '''
  This function creates the model and solves the parking problem.
  The input is a list of the car lengths.
  The output is the model (m) and the result object of the solution (status).
  '''
  CAR_LENGTH = [4, 4.5, 5, 4.1, 2.4, 5.2, 3.7, 3.5, 3.2, 4.5, 2.3, 3.3, 3.8, 4.6, 3]
  #cenary one, umlimited capacity, without an upper bound for capacity
  CAPACITY_1 = [1000, 1000]
  #cenary two, capacity of 15m for one side of the street
  CAPACITY_2 = [1000, 15]

  T = sum(CAR_LENGTH)

  # Create model
  m = ConcreteModel()

  #cars
  I = list(range(0,15))
  #street side
  J = [0,1] # 'left', 'right'

  m.I = Set(dimen=1,initialize=I)
  m.J = Set(dimen=1,initialize=J)

  # Variables
  m.x = Var(m.I, m.J, within=Binary)
  m.y = Var(m.J, within=Binary)

  #Setting the objective function, minimize the length
  m.OBJ = Objective(
      expr = sum(CAR_LENGTH[i] * m.x[i,0] for i in m.I ) - sum(CAR_LENGTH[i] * m.x[i,1] for i in m.I ),
      sense = minimize
  )

  #Setting all the constraints
  m.CONS1 = Constraint(m.J, rule = capacity_restriction)
  m.CONS2 = Constraint(rule = diff_pos(m,CAR_LENGTH))
  m.CONS3 = Constraint(m.I, rule = side_selected)

  ##Call the solver glpk
  status = SolverFactory('glpk',executable='/usr/bin/glpsol').solve(m)

  return status, m


In [71]:
status, m = parking_length()

##### **Results**

For unconstraint capacity, the cars are parked almost equaly in both sides.

In [73]:
# Print the status of the solved LP
print("Status = %s" % status.solver.termination_condition)

# Print the value of the variables at the optimum
for i in m.I:
  for j in m.J:
    if value(m.y[j]) != 0:
      print("%s = %f" % (m.x[i,j], value(m.x[i,j])))
      # print("%s = %f" % (m.y[j], value(m.y[j])))


# Print the value of the objective
print("Objective = %f" % value(m.OBJ))

Status = optimal
x[0,0] = 1.000000
x[0,1] = 0.000000
x[1,0] = 0.000000
x[1,1] = 1.000000
x[2,0] = 0.000000
x[2,1] = 1.000000
x[3,0] = 0.000000
x[3,1] = 1.000000
x[4,0] = 0.000000
x[4,1] = 1.000000
x[5,0] = 1.000000
x[5,1] = 0.000000
x[6,0] = 0.000000
x[6,1] = 1.000000
x[7,0] = 0.000000
x[7,1] = 1.000000
x[8,0] = 1.000000
x[8,1] = 0.000000
x[9,0] = 1.000000
x[9,1] = 0.000000
x[10,0] = 0.000000
x[10,1] = 1.000000
x[11,0] = 1.000000
x[11,1] = 0.000000
x[12,0] = 1.000000
x[12,1] = 0.000000
x[13,0] = 1.000000
x[13,1] = 0.000000
x[14,0] = 0.000000
x[14,1] = 1.000000
Objective = 0.100000


In [75]:
left=0
for i in m.I:
  if value(m.x[i,0]) != 0:
      print("%s = %f" % (m.x[i,0], CAR_LENGTH[i] * value(m.x[i,0])))
      left +=  CAR_LENGTH[i] * value(m.x[i,0])
print("\n---------------------")
print("Length left %f: " % left)
print("\n============================")

right=0
for i in m.I:
  if value(m.x[i,1]) != 0:
      print("%s = %f" % (m.x[i,1],  CAR_LENGTH[i] * value(m.x[i,1])))
      right +=  CAR_LENGTH[i] * value(m.x[i,1])
print("\n---------------------")
print("Length right %f: " % right)

x[0,0] = 4.000000
x[5,0] = 5.200000
x[8,0] = 3.200000
x[9,0] = 4.500000
x[11,0] = 3.300000
x[12,0] = 3.800000
x[13,0] = 4.600000

---------------------
Length left 28.600000: 

x[1,1] = 4.500000
x[2,1] = 5.000000
x[3,1] = 4.100000
x[4,1] = 2.400000
x[6,1] = 3.700000
x[7,1] = 3.500000
x[10,1] = 2.300000
x[14,1] = 3.000000

---------------------
Length right 28.500000: 


For constraint capacity, with 15m for one of the road sides

In [87]:
def capacity_restriction(m, j):
  '''
  This function represents the constraints of the road side capacity.
  The input isthe lenght of cars and the capacity per side of the road.
  '''
  temp = sum(CAR_LENGTH[i] * m.x[i,j] for i in m.I) - CAPACITY_2[j] * m.y[j]
  if type(temp) is int:    # This will be true if the sum was empty
    return Constraint.Skip
  return temp <= 0

In [88]:
status, m = parking_length()

In [89]:
# Print the status of the solved LP
print("Status = %s" % status.solver.termination_condition)

# Print the value of the variables at the optimum
for i in m.I:
  for j in m.J:
    if value(m.y[j]) != 0:
      print("%s = %f" % (m.x[i,j], value(m.x[i,j])))
      # print("%s = %f" % (m.y[j], value(m.y[j])))


# Print the value of the objective
print("Objective = %f" % value(m.OBJ))

Status = optimal
x[0,0] = 1.000000
x[0,1] = 0.000000
x[1,0] = 1.000000
x[1,1] = 0.000000
x[2,0] = 0.000000
x[2,1] = 1.000000
x[3,0] = 1.000000
x[3,1] = 0.000000
x[4,0] = 0.000000
x[4,1] = 1.000000
x[5,0] = 1.000000
x[5,1] = 0.000000
x[6,0] = 1.000000
x[6,1] = 0.000000
x[7,0] = 1.000000
x[7,1] = 0.000000
x[8,0] = 1.000000
x[8,1] = 0.000000
x[9,0] = 1.000000
x[9,1] = 0.000000
x[10,0] = 1.000000
x[10,1] = 0.000000
x[11,0] = 1.000000
x[11,1] = 0.000000
x[12,0] = 1.000000
x[12,1] = 0.000000
x[13,0] = 0.000000
x[13,1] = 1.000000
x[14,0] = 0.000000
x[14,1] = 1.000000
Objective = 27.100000


In [90]:
left=0
for i in m.I:
  if value(m.x[i,0]) != 0:
      print("%s = %f" % (m.x[i,0], CAR_LENGTH[i] * value(m.x[i,0])))
      left +=  CAR_LENGTH[i] * value(m.x[i,0])
print("\n---------------------")
print("Length left %f: " % left)
print("\n============================")

right=0
for i in m.I:
  if value(m.x[i,1]) != 0:
      print("%s = %f" % (m.x[i,1],  CAR_LENGTH[i] * value(m.x[i,1])))
      right +=  CAR_LENGTH[i] * value(m.x[i,1])
print("\n---------------------")
print("Length right %f: " % right)

x[0,0] = 4.000000
x[1,0] = 4.500000
x[3,0] = 4.100000
x[5,0] = 5.200000
x[6,0] = 3.700000
x[7,0] = 3.500000
x[8,0] = 3.200000
x[9,0] = 4.500000
x[10,0] = 2.300000
x[11,0] = 3.300000
x[12,0] = 3.800000

---------------------
Length left 42.100000: 

x[2,1] = 5.000000
x[4,1] = 2.400000
x[13,1] = 4.600000
x[14,1] = 3.000000

---------------------
Length right 15.000000: 


#### **Heuristic problem**

Heuristics are procedures for finding a solution based on rules that do not guarantee that the optimum will be reached. A heuristic for this problem could be first fit decreasing (FFD) or best fit decreasing (BFD). FFD consists of arranging the elements in non-increasing order of their size, and then for each element, trying to insert it into the first open bin, the space where it fits; if no such space exists, then open a new space and insert the element there.

In [194]:
CAR_LENGTH = [4, 4.5, 5, 4.1, 2.4, 5.2, 3.7, 3.5, 3.2, 4.5, 2.3, 3.3, 3.8, 4.6, 3]

In [195]:
def FFD(r, CAPACITY):
  '''
  This function is the first-fit decreasing (FFD) arranging the items in non-increasing
  order of their size, and then for each item try inserting it in the first open bin where it fits.
  '''
  remain = [CAPACITY]
  sol = [[]]
  for item in sorted(r, reverse=True):
      for j,free in enumerate(remain):
          if free >= item:
              remain[j] -= item
              sol[j].append(item)
              break
      else:
          sol.append([item])
          remain.append(B-item)
  return sol

In [196]:
def solver(r,CAPACITY):
  '''
  This function uses the output of first-fit decreasing (FFD) to solve the problem of
  minimizing the occupied length of the road.
  '''
  #Start with a big value for objective function
  M=1000
  sol = FFD(r, CAPACITY)
  for s in sol:
    total_cars = [4, 4.5, 5, 4.1, 2.4, 5.2, 3.7, 3.5, 3.2, 4.5, 2.3, 3.3, 3.8, 4.6, 3]
    #calculate the total size of first elements from FFD
    S=sum(s)
    for i in s:
      total_cars.pop(total_cars.index(i))
    #calculate the total size of elements without the group found by FFD
    T=sum(total_cars)
    #calculate the new value of the objective function
    obj = abs(T-S)
    #In case is lower, update the value and continue until find the minimum
    if obj < M:
      M = abs(T-S)
      left = s
      right = total_cars
    else:
      continue
  return left, right, M

In [198]:
#Result for unlimited capacity
left, right, Obj = solver(CAR_LENGTH,CAPACITY=1000)
print("The cars parked on left side are: %s" %(left))
print("Length left %f: " % sum(left))
print("\n---------------------")
print("The cars parked on rigth side are: %s" %(right))
print("Length right %f: " % sum(right))
print("\n---------------------")
print("The Objective function value is %.2f " %(Obj))

The cars parked on left side are: [5.2, 5, 4.6, 4.5, 4.5, 4.1, 4, 3.8, 3.7, 3.5, 3.3, 3.2, 3, 2.4, 2.3]
Length left 57.100000: 

---------------------
The cars parked on rigth side are: []
Length right 0.000000: 

---------------------
The Objective function value is 57.10 


In [199]:
#Result for a limited capacity of 15.
left, right, Obj = solver(CAR_LENGTH,CAPACITY=15)
print("The cars parked on left side are: %s" %(left))
print("Length left %f: " % sum(left))
print("\n---------------------")
print("The cars parked on rigth side are: %s" %(right))
print("Length right %f: " % sum(right))
print("\n---------------------")
print("The Objective function value is %.2f " %(Obj))

The cars parked on left side are: [5.2, 5, 4.6]
Length left 14.800000: 

---------------------
The cars parked on rigth side are: [4, 4.5, 4.1, 2.4, 3.7, 3.5, 3.2, 4.5, 2.3, 3.3, 3.8, 3]
Length right 42.300000: 

---------------------
The Objective function value is 27.50 


### **REFERENCE**

* Mathematical Optimization: Solving Problems using SCIP and Python https://scipbook.readthedocs.io/en/latest/bpp.html

* P. C. Gilmore and R. E. Gomory. A linear programming approach to the cutting-stock problem. Operations Research, 9(6):849–859, 1961
https://www.researchgate.net/publication/266478800_A_Linear_Programming_Approach_to_the_Cutting_Stock_Problem_I

* P. C. Gilmore and R. E. Gomory. A linear programming approach to the cutting stock problem–Part II. Operations Research, 11(6):863–888, 1963.
https://www.researchgate.net/publication/265461353_A_Linear_Programming_Approach_to_the_Cutting_Stock_Problem-Part_II


* Pyomo online documentation http://pyomo.readthedocs.io/

* GLPK online documentation https://www.gnu.org/software/glpk/#TOCdocumentation